In [1]:
#importar bibliotecas
import xmltodict
import os
import json #apenas para visualizar e tratar erros
import pandas as pd

#buscar arquivos e ler informações
def pegar_infos(nome_arquivo, valores):
    #print(f'Pegou as informações {nome_arquivo}')
    with open(f'nfs/{nome_arquivo}', 'rb') as arquivo_xml:
        dic_arquivo = xmltodict.parse(arquivo_xml)
        #print(json.dumps(dic_arquivo, indent=4))
        try: #dentro do try para tratar possiveis erros            
            infos_nf = dic_arquivo['nfeProc']['NFe']['infNFe']              
            numero_nota = infos_nf['ide']['nNF']
            data_nota = infos_nf['ide']['dhEmi']
            cnpj_emissora = infos_nf['emit']['CNPJ']
            empresa_emissora = infos_nf['emit']['xNome']
            cfop = infos_nf['det']['prod']['CFOP']
            valor_nf = infos_nf['total']['ICMSTot']['vNF']
            pis = infos_nf['total']['ICMSTot']['vPIS']
            cofins = infos_nf['total']['ICMSTot']['vCOFINS']
            icms = infos_nf['total']['ICMSTot']['vICMS']
            valores.append([numero_nota, data_nota, cnpj_emissora, empresa_emissora, cfop, valor_nf, pis, cofins, icms])
        except Exception as e:
#             print(e)
#             print(json.dumps(dic_arquivo, indent=4))
            pass
    
lista_arquivos = os.listdir('nfs')

#criar tabela
colunas = ['numero_nota', 'data_nota', 'cnpj_emissora', 'empresa_emissora', 'cfop', 'valor_nf', 'pis', 'cofins', 'icms']
valores = []

for arquivo in lista_arquivos:
    pegar_infos(arquivo, valores)
  
    

tabela = pd.DataFrame(columns=colunas, data=valores)

#Editar Tabela

#Mudar Type object para float
tabela_type_float = ['valor_nf', 'pis', 'cofins', 'icms']
tabela[tabela_type_float] = tabela[tabela_type_float].astype(float)

#arrendor valores para 2 casas decimais
tabela_arred_2 = ['valor_nf', 'pis', 'cofins', 'icms']
tabela[tabela_arred_2] = tabela[tabela_arred_2].round(2)

tabela['data_nota'] = tabela['data_nota'].str.slice(0, 10)
#display(tabela)
#print(tabela.dtypes)
      
#Transformar tabela em Excel
tabela.to_excel('Notas Fiscais.xlsx', index=False)